In [ ]:
!pip install transformers sentence-transformers pdfplumber gradio pinecone-client cohere fitz finnlp



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pdfplumber
import pinecone
import cohere
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import gradio as gr

In [ ]:
def extract_pl_data(pdf_path):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_text += page.extract_text()
    return extracted_text
pdf_path = "/content/drive/MyDrive/Colab Notebooks/RAG/Sample Financial Statement (1).pdf"
text = extract_pl_data(pdf_path)
print(text[:500])


INFOSYS LIMITED AND SUBSIDIARIES
Condensed Consolidated Financial Statements under
Indian Accounting Standards (Ind AS)
for the three months and year ended March 31, 2024
Index Page No.
Condensed Consolidated Balance Sheet ………………………………………………………………………………………………1 ……………………..
Condensed Consolidated Statement of Profit and Loss ………………………………………………………………………………2……………………………………..
Condensed Consolidated Statement of Changes in Equity ……………………………………………………………………………3………………………………………..
Condensed Consolidated Sta


In [ ]:
!pip uninstall pymupdf -y
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 75.7 MB/s eta 0:00:00


In [ ]:
import fitz
import pymupdf


In [ ]:
import os
import fitz
from pinecone import Pinecone, ServerlessSpec
import cohere


PINECONE_API = "pcsk_3b1NXB_SVGhgGavGiujWA2tj1uCG7KcKiZRN8nCLoLzAjP8cr86i4aEz4sAdhpXepg1fpf"
PINECONE_INDEX = "plrag"
PINECONE_HOST = "https://plrag-hd29a9a.svc.aped-4627-b74a.pinecone.io"
COHERE_API_ = "d928nahA8yF01VaknHiDUV92sYb4rwFpj5T5F0Zv"

pc = Pinecone(api_key=PINECONE_API)
index = pc.Index(
    PINECONE_INDEX_NAME,
    spec=ServerlessSpec(cloud="aws", region="us-west-1")
)
print(f"Connected to Pinecone index: {PINECONE_INDEX}")

co = cohere.Client(COHERE_API_KEY)
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text
def generate_embeddings_and_upsert(pdf_path):
    text_data = extract_text_from_pdf(pdf_path)
    if not text_data.strip():
        print("No text extracted from the PDF.")
        return
    chunks = [text_data[i:i+500] for i in range(0, len(text_data), 500)]

    response = co.embed(texts=chunks, model="embed-english-v3.0", input_type="search_document")
    embeddings = response.embeddings
    if len(embeddings) == 0:
        print("Error: No embeddings generated.")
        return
    print(f"Generated {len(embeddings)} embeddings, embedding size: {len(embeddings[0])}")
    vectors = [(f"id_{i}", embeddings[i], {"text": chunks[i]}) for i in range(len(chunks))]
    index.upsert(vectors)
    print(f"Upserted {len(chunks)} records into Pinecone.")
pdf_file_path = "/content/drive/MyDrive/Colab Notebooks/RAG/Sample Financial Statement (1).pdf"
generate_embeddings_and_upsert(pdf_file_path)


Connected to Pinecone index: plrag
Generated 437 embeddings, embedding size: 1024
Upserted 437 records into Pinecone.


In [ ]:
import cohere
from pinecone import Pinecone
from transformers import pipeline

# Pinecone configuration
PINECONE_API_KEY = "pcsk_3b1NXB_SVGhgGavGiujWA2tj1uCG7KcKiZRN8nCLoLzAjP8cr86i4aEz4sAdhpXepg1fpf"
PINECONE_INDEX_NAME = "plrag"
PINECONE_HOST = "https://plrag-hd29a9a.svc.aped-4627-b74a.pinecone.io"
COHERE_API_KEY = "d928nahA8yF01VaknHiDUV92sYb4rwFpj5T5F0Zv"


# Initialize Pinecone connection
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize Cohere client
co = cohere.Client(COHERE_API_KEY)

def query_pinecone(query_text, top_k=5):
    """
    Queries Pinecone to retrieve relevant financial data based on user input.
    """
    # Generate embedding for the query
    query_embedding = co.embed(texts=[query_text], model="embed-english-v3.0", input_type="search_query").embeddings[0]

    # Query Pinecone index
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    if not results['matches']:
        print("No relevant documents found.")
        return []

    retrieved_texts = [match['metadata']['text'] for match in results['matches']]
    return retrieved_texts

def generate_answer(query_text):
    """
    Generates an answer based on retrieved data using a Hugging Face model.
    """
    # Query Pinecone for relevant data
    retrieved_texts = query_pinecone(query_text)

    if not retrieved_texts:
        return "Sorry, I couldn't find relevant information."

    # Concatenate retrieved chunks
    context = " ".join(retrieved_texts)

    # Load Hugging Face QA pipeline
    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

    # Generate answer using retrieved context
    response = qa_pipeline(question=query_text, context=context)
    return response['answer']

# Example query
query_text = "What is the net income for Q3 2024?"
answer = generate_answer(query_text)
print("Answer:", answer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Answer: 2,071


In [ ]:
import cohere
from pinecone import Pinecone
from transformers import pipeline

# Pinecone configuration
PINECONE_API_KEY = "pcsk_3b1NXB_SVGhgGavGiujWA2tj1uCG7KcKiZRN8nCLoLzAjP8cr86i4aEz4sAdhpXepg1fpf"
PINECONE_INDEX_NAME = "plrag"
PINECONE_HOST = "https://plrag-hd29a9a.svc.aped-4627-b74a.pinecone.io"
COHERE_API_KEY = "d928nahA8yF01VaknHiDUV92sYb4rwFpj5T5F0Zv"


# Initialize Pinecone connection
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize Cohere client
co = cohere.Client(COHERE_API_KEY)

def query_pinecone(query_text, top_k=5):
    """
    Queries Pinecone to retrieve relevant financial data based on user input.
    """
    # Generate embedding for the query
    query_embedding = co.embed(texts=[query_text], model="embed-english-v3.0", input_type="search_query").embeddings[0]

    # Query Pinecone index
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    if not results['matches']:
        print("No relevant documents found.")
        return []

    retrieved_texts = [match['metadata']['text'] for match in results['matches']]
    return retrieved_texts

def generate_answer(query_text):
    """
    Generates an answer based on retrieved data using a Hugging Face model.
    """
    # Query Pinecone for relevant data
    retrieved_texts = query_pinecone(query_text)

    if not retrieved_texts:
        return "Sorry, I couldn't find relevant information."

    # Concatenate retrieved chunks
    context = " ".join(retrieved_texts)

    # Load Hugging Face QA pipeline
    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

    # Generate answer using retrieved context
    response = qa_pipeline(question=query_text, context=context)
    return response['answer']

# Example query
query_text = "What is the EBITDA for the last quarter?"
answer = generate_answer(query_text)
print("Answer:", answer)


Device set to use cpu


Answer: 
X14AO
28



In [ ]:
query_text = "What is the EBITDA for the last quarter?"
answer = generate_answer(query_text)
print("Answer:", answer)


Device set to use cpu


Answer: 
X14AO
28



In [ ]:
query_text = "What were the major financial highlights of Q4 2024?"
answer = generate_answer(query_text)
print("Answer:", answer)
print("************************************")
query_text = "Summarize the P&L statement for the year 2023"
answer = generate_answer(query_text)
print("Answer:", answer)
print("************************************")
query_text = "How does the debt-to-equity ratio compare with previous years?"
answer = generate_answer(query_text)
print("Answer:", answer)
print("************************************")
query_text = "What is the free cash flow for the year?"
answer = generate_answer(query_text)
print("Answer:", answer)
print("************************************")
query_text = "How do operating and non-operating incomes compare for Q3 2024?"
answer = generate_answer(query_text)
print("Answer:", answer)
print("************************************")
query_text = "Breakdown of cost of goods sold (COGS) for the year?"
answer = generate_answer(query_text)
print("Answer:", answer)


Device set to use cpu


Answer: 
2024
2023
2024
2023
************************************


Device set to use cpu


Answer: 
38
2.24
************************************


Device set to use cpu


...

Answer: net cash provided by operating activities less capital expenditure
************************************


Device set to use cpu


Answer: 
2024
2023
************************************


Device set to use cpu


Answer: 2023


In [ ]:
import cohere
from pinecone import Pinecone
from transformers import pipeline

# Pinecone configuration
PINECONE_API_KEY = "pcsk_3b1NXB_SVGhgGavGiujWA2tj1uCG7KcKiZRN8nCLoLzAjP8cr86i4aEz4sAdhpXepg1fpf"
PINECONE_INDEX_NAME = "plrag"
PINECONE_HOST = "https://plrag-hd29a9a.svc.aped-4627-b74a.pinecone.io"
COHERE_API_KEY = "d928nahA8yF01VaknHiDUV92sYb4rwFpj5T5F0Zv"


# Initialize Pinecone connection
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize Cohere client
co = cohere.Client(COHERE_API_KEY)

def query_pinecone(query_text, top_k=5):
    """
    Queries Pinecone to retrieve relevant financial data based on user input.
    """
    # Generate embedding for the query
    query_embedding = co.embed(texts=[query_text], model="embed-english-v3.0", input_type="search_query").embeddings[0]

    # Query Pinecone index
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    if not results['matches']:
        print("No relevant documents found.")
        return []

    retrieved_texts = [match['metadata']['text'] for match in results['matches']]
    return retrieved_texts

def generate_answer(query_text):
    """
    Generates an answer based on retrieved data using a Hugging Face model.
    """
    # Query Pinecone for relevant data
    retrieved_texts = query_pinecone(query_text)

    if not retrieved_texts:
        return "Sorry, I couldn't find relevant information."

    # Concatenate retrieved chunks
    context = " ".join(retrieved_texts)

    # Load Hugging Face QA pipeline
    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

    # Generate answer using retrieved context
    response = qa_pipeline(question=query_text, context=context)
    return response['answer']

# Loop to ask questions until the user decides to exit
while True:
    query_text = input("Enter a financial question (or type 'exit' to quit): ")

    if query_text.lower() == 'exit':
        print("Exiting the program. Goodbye!")
        break

    answer = generate_answer(query_text)
    print(f"Answer: {answer}\n")


Enter a financial question (or type 'exit' to quit): Provide insights on cost reduction initiatives.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

...

Answer: 50

Enter a financial question (or type 'exit' to quit): "Compare gross margin percentage between 2023 and 2024."


Device set to use cuda:0


Answer: 1,432

Enter a financial question (or type 'exit' to quit): "How does subscription revenue compare to one-time sales?"


Device set to use cuda:0


Answer: 54% and 52%,

Enter a financial question (or type 'exit' to quit): exit
Exiting the program. Goodbye!


In [ ]:
import gradio as gr
import cohere
from pinecone import Pinecone
from transformers import pipeline

# Pinecone configuration
PINECONE_API_KEY = "pcsk_3b1NXB_SVGhgGavGiujWA2tj1uCG7KcKiZRN8nCLoLzAjP8cr86i4aEz4sAdhpXepg1fpf"
PINECONE_INDEX_NAME = "plrag"
PINECONE_HOST = "https://plrag-hd29a9a.svc.aped-4627-b74a.pinecone.io"
COHERE_API_KEY = "d928nahA8yF01VaknHiDUV92sYb4rwFpj5T5F0Zv"

# Initialize Pinecone connection
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize Cohere client
co = cohere.Client(COHERE_API_KEY)

def query_pinecone(query_text, top_k=5):
    """
    Queries Pinecone to retrieve relevant financial data based on user input.
    """
    # Generate embedding for the query
    query_embedding = co.embed(texts=[query_text], model="embed-english-v3.0", input_type="search_query").embeddings[0]

    # Query Pinecone index
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    if not results['matches']:
        return "No relevant documents found."

    retrieved_texts = [match['metadata']['text'] for match in results['matches']]
    return " ".join(retrieved_texts)

def generate_answer(query_text):
    """
    Generates an answer based on retrieved data using a Hugging Face model.
    """
    # Query Pinecone for relevant data
    retrieved_texts = query_pinecone(query_text)
    if not retrieved_texts:
        return "Sorry, I couldn't find relevant information."

    context = retrieved_texts
    # Use Hugging Face QA pipeline to generate the answer from context
    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
    response = qa_pipeline(question=query_text, context=context)
    return response['answer']

# Gradio function to handle user query input and display answer
def handle_query(query):
    answer = generate_answer(query)
    return answer

# Gradio UI for query input
iface = gr.Interface(
    fn=handle_query,  # Function to handle query input and provide answer
    inputs=[gr.Textbox(label="Ask a financial question")],  # Textbox for user input
    outputs=[gr.Textbox(label="Answer")],  # Output to show the generated answer
    live=True  # Allow live updates while interacting
)

# Launch the Gradio interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c45505a8d8f3aa3160.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
